In [1]:
#DataTools.py
""" Module for the data loading pipeline for the model to train """


def get_transform(new_size=None):
    """
    obtain the image transforms required for the input data
    :param new_size: size of the resized images
    :return: image_transform => transform object from TorchVision
    """
    from torchvision.transforms import ToTensor, Normalize, Compose, Resize

    if new_size is not None:
        image_transform = Compose([
            Resize(new_size),
            ToTensor(),
            Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])

    else:
        image_transform = Compose([
            ToTensor(),
            Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])
    return image_transform


def get_data_loader(dataset, batch_size, num_workers):
    """
    generate the data_loader from the given dataset
    :param dataset: dataset for training (Should be a PyTorch dataset)
                    Make sure every item is an Image
    :param batch_size: batch size of the data
    :param num_workers: num of parallel readers
    :return: dl => dataloader for the dataset
    """
    from torch.utils.data import DataLoader

    dl = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    )

    return dl

In [2]:
#Losses.py
""" Module implementing various loss functions """

import torch as th



# =============================================================
# Normal versions of the Losses:
# =============================================================



class WGAN_GP:

    def __init__(self, dis, drift=0.001, use_gp=False):
        self.dis=dis
        self.drift = drift
        self.use_gp = use_gp

    def __gradient_penalty(self, real_samps, fake_samps,
                           height, alpha, reg_lambda=10):
        """
        private helper for calculating the gradient penalty
        :param real_samps: real samples
        :param fake_samps: fake samples
        :param height: current depth in the optimization
        :param alpha: current alpha for fade-in
        :param reg_lambda: regularisation lambda
        :return: tensor (gradient penalty)
        """
        batch_size = real_samps.shape[0]

        # generate random epsilon
        epsilon = th.rand((batch_size, 1, 1, 1)).to(fake_samps.device)

        # create the merge of both real and fake samples
        merged = epsilon * real_samps + ((1 - epsilon) * fake_samps)
        merged.requires_grad_(True)

        # forward pass
        op = self.dis(merged, height, alpha)

        # perform backward pass from op to merged for obtaining the gradients
        gradient = th.autograd.grad(outputs=op, inputs=merged,
                                    grad_outputs=th.ones_like(op), create_graph=True,
                                    retain_graph=True, only_inputs=True)[0]

        # calculate the penalty using these gradients
        gradient = gradient.view(gradient.shape[0], -1)
        penalty = reg_lambda * ((gradient.norm(p=2, dim=1) - 1) ** 2).mean()

        # return the calculated penalty:
        return penalty

    def dis_loss(self, real_samps, fake_samps, height, alpha):
        # define the (Wasserstein) loss
        fake_out = self.dis(fake_samps, height, alpha)
        real_out = self.dis(real_samps, height, alpha)

        loss = (th.mean(fake_out) - th.mean(real_out)
                + (self.drift * th.mean(real_out ** 2)))

        # calculate the WGAN-GP (gradient penalty)
        gp = self.__gradient_penalty(real_samps, fake_samps, height, alpha)
        loss += gp

        return loss

    def gen_loss(self, _, fake_samps, height, alpha):
        # calculate the WGAN loss for generator
        loss = -th.mean(self.dis(fake_samps, height, alpha))

        return loss


In [3]:
#CustomLayers.py
""" Module containing custom layers """

import torch as th


# extending Conv2D and Deconv2D layers for equalized learning rate logic
class _equalized_conv2d(th.nn.Module):
    """ conv2d with the concept of equalized learning rate
        Args:
            :param c_in: input channels
            :param c_out:  output channels
            :param k_size: kernel size (h, w) should be a tuple or a single integer
            :param stride: stride for conv
            :param pad: padding
            :param bias: whether to use bias or not
    """

    def __init__(self, c_in, c_out, k_size, stride=1, pad=0, bias=True):
        """ constructor for the class """
        from torch.nn.modules.utils import _pair
        from numpy import sqrt, prod

        super(_equalized_conv2d, self).__init__()

        # define the weight and bias if to be used
        self.weight = th.nn.Parameter(th.nn.init.normal_(
            th.empty(c_out, c_in, *_pair(k_size))
        ))

        self.use_bias = bias
        self.stride = stride
        self.pad = pad

        if self.use_bias:
            self.bias = th.nn.Parameter(th.FloatTensor(c_out).fill_(0))

        fan_in = prod(_pair(k_size)) * c_in  # value of fan_in
        self.scale = sqrt(2) / sqrt(fan_in)

    def forward(self, x):
        """
        forward pass of the network
        :param x: input
        :return: y => output
        """
        from torch.nn.functional import conv2d

        return conv2d(input=x,
                      weight=self.weight * self.scale,  # scale the weight on runtime
                      bias=self.bias if self.use_bias else None,
                      stride=self.stride,
                      padding=self.pad)

    def extra_repr(self):
        return ", ".join(map(str, self.weight.shape))


class _equalized_deconv2d(th.nn.Module):
    """ Transpose convolution using the equalized learning rate
        Args:
            :param c_in: input channels
            :param c_out: output channels
            :param k_size: kernel size
            :param stride: stride for convolution transpose
            :param pad: padding
            :param bias: whether to use bias or not
    """

    def __init__(self, c_in, c_out, k_size, stride=1, pad=0, bias=True):
        """ constructor for the class """
        from torch.nn.modules.utils import _pair
        from numpy import sqrt

        super(_equalized_deconv2d, self).__init__()

        # define the weight and bias if to be used
        self.weight = th.nn.Parameter(th.nn.init.normal_(
            th.empty(c_in, c_out, *_pair(k_size))
        ))

        self.use_bias = bias
        self.stride = stride
        self.pad = pad

        if self.use_bias:
            self.bias = th.nn.Parameter(th.FloatTensor(c_out).fill_(0))

        fan_in = c_in  # value of fan_in for deconv
        self.scale = sqrt(2) / sqrt(fan_in)

    def forward(self, x):
        """
        forward pass of the layer
        :param x: input
        :return: y => output
        """
        from torch.nn.functional import conv_transpose2d

        return conv_transpose2d(input=x,
                                weight=self.weight * self.scale,  # scale the weight on runtime
                                bias=self.bias if self.use_bias else None,
                                stride=self.stride,
                                padding=self.pad)

    def extra_repr(self):
        return ", ".join(map(str, self.weight.shape))




# ----------------------------------------------------------------------------
# Pixelwise feature vector normalization.
# reference: https://github.com/tkarras/progressive_growing_of_gans/blob/master/networks.py#L120
# ----------------------------------------------------------------------------
class PixelwiseNorm(th.nn.Module):
    def __init__(self):
        super(PixelwiseNorm, self).__init__()

    def forward(self, x, alpha=1e-8):
        """
        forward pass of the module
        :param x: input activations volume
        :param alpha: small number for numerical stability
        :return: y => pixel normalized activations
        """
        y = x.pow(2.).mean(dim=1, keepdim=True).add(alpha).sqrt()  # [N1HW]
        y = x / y  # normalize the input x volume
        return y


# ==========================================================
# Layers required for Building The generator and
# discriminator
# ==========================================================
class GenInitialBlock(th.nn.Module):
    """ Module implementing the initial block of the input """

    def __init__(self, in_channels):
        """
        constructor for the inner class
        :param in_channels: number of input channels to the block
        """
        from torch.nn import LeakyReLU

        super(GenInitialBlock, self).__init__()

        self.conv_1 = _equalized_deconv2d(in_channels, in_channels, (4, 4), bias=True,pad=1)
        
        self.conv_2 = _equalized_conv2d(in_channels, in_channels, (3, 3), pad=1, bias=True)

        # Pixelwise feature vector normalization operation
        self.pixNorm = PixelwiseNorm()

        # leaky_relu:
        self.lrelu = LeakyReLU(0.2)

    def forward(self, x):
        """
        forward pass of the block
        :param x: input to the module
        :return: y => output
        """
        # convert the tensor shape:
        y = th.unsqueeze(th.unsqueeze(x, -1), -1)

        # perform the forward computations:
        y = self.lrelu(self.conv_1(y))
        y = self.lrelu(self.conv_2(y))

        # apply pixel norm
        y = self.pixNorm(y)

        return y


class GenGeneralConvBlock(th.nn.Module):
    """ Module implementing a general convolutional block """

    def __init__(self, in_channels, out_channels):
        """
        constructor for the class
        :param in_channels: number of input channels to the block
        :param out_channels: number of output channels required
        """
        from torch.nn import LeakyReLU
        from torch.nn.functional import interpolate

        super(GenGeneralConvBlock, self).__init__()

        self.upsample = lambda x: interpolate(x, scale_factor=2)

        self.conv_1 = _equalized_conv2d(in_channels, out_channels, (3, 3),
                                        pad=1, bias=True)
        self.conv_2 = _equalized_conv2d(out_channels, out_channels, (3, 3),
                                        pad=1, bias=True)

        # Pixelwise feature vector normalization operation
        self.pixNorm = PixelwiseNorm()

        # leaky_relu:
        self.lrelu = LeakyReLU(0.2)

    def forward(self, x):
        """
        forward pass of the block
        :param x: input
        :return: y => output
        """
        y = self.upsample(x)
        y = self.pixNorm(self.lrelu(self.conv_1(y)))
        y = self.pixNorm(self.lrelu(self.conv_2(y)))

        return y


# function to calculate the Exponential moving averages for the Generator weights
# This function updates the exponential average weights based on the current training
def update_average(model_tgt, model_src, beta):
    """
    update the model_target using exponential moving averages
    :param model_tgt: target model
    :param model_src: source model
    :param beta: value of decay beta
    :return: None (updates the target model)
    """

    # utility function for toggling the gradient requirements of the models
    def toggle_grad(model, requires_grad):
        for p in model.parameters():
            p.requires_grad_(requires_grad)

    # turn off gradient calculation
    toggle_grad(model_tgt, False)
    toggle_grad(model_src, False)

    param_dict_src = dict(model_src.named_parameters())

    for p_name, p_tgt in model_tgt.named_parameters():
        p_src = param_dict_src[p_name]
        assert (p_src is not p_tgt)
        p_tgt.copy_(beta * p_tgt + (1. - beta) * p_src)

    # turn back on the gradient calculation
    toggle_grad(model_tgt, True)
    toggle_grad(model_src, True)


class MinibatchStdDev(th.nn.Module):
    """
    Minibatch standard deviation layer for the discriminator
    """

    def __init__(self):
        """
        derived class constructor
        """
        super(MinibatchStdDev, self).__init__()

    def forward(self, x, alpha=1e-8):
        """
        forward pass of the layer
        :param x: input activation volume
        :param alpha: small number for numerical stability
        :return: y => x appended with standard deviation constant map
        """
        batch_size, _, height, width = x.shape

        # [B x C x H x W] Subtract mean over batch.
        y = x - x.mean(dim=0, keepdim=True)

        # [1 x C x H x W]  Calc standard deviation over batch
        y = th.sqrt(y.pow(2.).mean(dim=0, keepdim=False) + alpha)

        # [1]  Take average over feature_maps and pixels.
        y = y.mean().view(1, 1, 1, 1)

        # [B x 1 x H x W]  Replicate over group and pixels.
        y = y.repeat(batch_size, 1, height, width)

        # [B x C x H x W]  Append as new feature_map.
        y = th.cat([x, y], 1)

        # return the computed values:
        return y


class DisFinalBlock(th.nn.Module):
    """ Final block for the Discriminator """

    def __init__(self, in_channels):
        """
        constructor of the class
        :param in_channels: number of input channels
        """
        from torch.nn import LeakyReLU

        super(DisFinalBlock, self).__init__()

        # declare the required modules for forward pass
        self.batch_discriminator = MinibatchStdDev()

        self.conv_1 = _equalized_conv2d(in_channels + 1, in_channels, (3, 3), pad=1, bias=True)
        self.conv_2 = _equalized_conv2d(in_channels, in_channels, (4, 4), pad=1, bias=True)
        # final conv layer emulates a fully connected layer
        self.conv_3 = _equalized_conv2d(in_channels, 1, (1, 1), bias=True)

        # leaky_relu:
        self.lrelu = LeakyReLU(0.2)

    def forward(self, x):
        """
        forward pass of the FinalBlock
        :param x: input
        :return: y => output
        """
        # minibatch_std_dev layer
        y = self.batch_discriminator(x)

        # define the computations
        y = self.lrelu(self.conv_1(y))
        y = self.lrelu(self.conv_2(y))

        # fully connected layer
        y = self.conv_3(y)  # This layer has linear activation

        # flatten the output raw discriminator scores
        return y.view(-1)



class DisGeneralConvBlock(th.nn.Module):
    """ General block in the discriminator  """

    def __init__(self, in_channels, out_channels):
        """
        constructor of the class
        :param in_channels: number of input channels
        :param out_channels: number of output channels
        """
        from torch.nn import AvgPool2d, LeakyReLU

        super(DisGeneralConvBlock, self).__init__()

        self.conv_1 = _equalized_conv2d(in_channels, in_channels, (3, 3), pad=1, bias=True)
        self.conv_2 = _equalized_conv2d(in_channels, out_channels, (3, 3), pad=1, bias=True)

        self.downSampler = AvgPool2d(2)

        # leaky_relu:
        self.lrelu = LeakyReLU(0.2)

    def forward(self, x):
        """
        forward pass of the module
        :param x: input
        :return: y => output
        """
        # define the computations
        y = self.lrelu(self.conv_1(x))
        y = self.lrelu(self.conv_2(y))
        y = self.downSampler(y)

        return y

In [ ]:
""" Module implementing GAN which will be trained using the Progressive growing
    technique -> https://arxiv.org/abs/1710.10196
"""
import datetime
import os
import time
import timeit
import copy
import numpy as np
import torch as th


# ========================================================================================
# Generator Module
# can be used with ProGAN, ConditionalProGAN or standalone (for inference)
# ========================================================================================

class Generator(th.nn.Module):
    """ Generator of the GAN network """

    def __init__(self, depth, latent_size):
        """
        constructor for the Generator class
        :param depth: required depth of the Network
        :param latent_size: size of the latent manifold
        """
        from torch.nn import ModuleList
        from torch.nn.functional import interpolate

        super(Generator, self).__init__()

        assert latent_size != 0 and ((latent_size & (latent_size - 1)) == 0), \
            "latent size not a power of 2"
        if depth >= 4:
            assert latent_size >= np.power(2, depth - 4), "latent size will diminish to zero"

        # state of the generator:
        self.depth = depth
        self.latent_size = latent_size

        # register the modules required for the GAN
        self.initial_block = GenInitialBlock(self.latent_size)

        # create a module list of the other required general convolution blocks
        self.layers = ModuleList([])  # initialize to empty list

        # create the ToRGB layers for various outputs:
        self.toRGB = lambda in_channels: \
            _equalized_conv2d(in_channels, 3, (1, 1), bias=True)

        self.rgb_converters = ModuleList([self.toRGB(self.latent_size)])

        # create the remaining layers
        for i in range(self.depth - 1):
            if i <= 2:
                layer = GenGeneralConvBlock(self.latent_size,
                                            self.latent_size)
                rgb = self.toRGB(self.latent_size)
            else:
                layer = GenGeneralConvBlock(
                    int(self.latent_size // np.power(2, i - 3)),
                    int(self.latent_size // np.power(2, i - 2))
                )
                rgb = self.toRGB(int(self.latent_size // np.power(2, i - 2)))
            self.layers.append(layer)
            self.rgb_converters.append(rgb)

        # register the temporary upsampler
        self.temporaryUpsampler = lambda x: interpolate(x, scale_factor=2)

    def forward(self, x, depth, alpha):
        """
        forward pass of the Generator
        :param x: input noise
        :param depth: current depth from where output is required
        :param alpha: value of alpha for fade-in effect
        :return: y => output
        """

        assert depth < self.depth, "Requested output depth cannot be produced"

        y = self.initial_block(x)

        if depth > 0:
            for block in self.layers[:depth - 1]:
                y = block(y)

            residual = self.rgb_converters[depth - 1](self.temporaryUpsampler(y))
            straight = self.rgb_converters[depth](self.layers[depth - 1](y))

            out = (alpha * straight) + ((1 - alpha) * residual)

        else:
            out = self.rgb_converters[0](y)

        return out


# ========================================================================================
# Discriminator Module
# can be used with ProGAN or standalone (for inference).
# Note this cannot be used with ConditionalProGAN
# ========================================================================================

class Discriminator(th.nn.Module):
    """ Discriminator of the GAN """

    def __init__(self, height, feature_size):
        """
        constructor for the class
        :param height: total height of the discriminator (Must be equal to the Generator depth)
        :param feature_size: size of the deepest features extracted
                             (Must be equal to Generator latent_size)
        """
        from torch.nn import ModuleList, AvgPool2d

        super(Discriminator, self).__init__()

        assert feature_size != 0 and ((feature_size & (feature_size - 1)) == 0), \
            "latent size not a power of 2"
        if height >= 4:
            assert feature_size >= np.power(2, height - 4), "feature size cannot be produced"

        # create state of the object
        self.height = height
        self.feature_size = feature_size

        self.final_block = DisFinalBlock(self.feature_size)

        # create a module list of the other required general convolution blocks
        self.layers = ModuleList([])  # initialize to empty list

        # create the fromRGB layers for various inputs:
        self.fromRGB = lambda out_channels: \
            _equalized_conv2d(3, out_channels, (1, 1), bias=True)
        self.rgb_to_features = ModuleList([self.fromRGB(self.feature_size)])

        # create the remaining layers
        for i in range(self.height - 1):
            if i > 2:
                layer = DisGeneralConvBlock(
                    int(self.feature_size // np.power(2, i - 2)),
                    int(self.feature_size // np.power(2, i - 3))
                )
                rgb = self.fromRGB(int(self.feature_size // np.power(2, i - 2)))
            else:
                layer = DisGeneralConvBlock(self.feature_size,
                                            self.feature_size)
                rgb = self.fromRGB(self.feature_size)

            self.layers.append(layer)
            self.rgb_to_features.append(rgb)

        # register the temporary downSampler
        self.temporaryDownsampler = AvgPool2d(2)

    def forward(self, x, height, alpha):
        """
        forward pass of the discriminator
        :param x: input to the network
        :param height: current height of operation (Progressive GAN)
        :param alpha: current value of alpha for fade-in
        :return: out => raw prediction values (WGAN-GP)
        """

        assert height < self.height, "Requested output depth cannot be produced"

        if height > 0:
            residual = self.rgb_to_features[height - 1](self.temporaryDownsampler(x))

            straight = self.layers[height - 1](
                self.rgb_to_features[height](x)
            )

            y = (alpha * straight) + ((1 - alpha) * residual)

            for block in reversed(self.layers[:height - 1]):
                y = block(y)
        else:
            y = self.rgb_to_features[0](x)

        out = self.final_block(y)

        return out

# ========================================================================================
# ProGAN Module (Unconditional)
# ========================================================================================

class ProGAN:
    """ Wrapper around the Generator and the Discriminator """

    def __init__(self, depth=7, latent_size=512, learning_rate=0.001, beta_1=0,
                 beta_2=0.99, eps=1e-8, drift=0.001, n_critic=1,
                 use_ema=True, ema_decay=0.999,
                 device=th.device("cpu")):
        """
        constructor for the class
        :param depth: depth of the GAN (will be used for each generator and discriminator)
        :param latent_size: latent size of the manifold used by the GAN
        :param learning_rate: learning rate for Adam
        :param beta_1: beta_1 for Adam
        :param beta_2: beta_2 for Adam
        :param eps: epsilon for Adam
        :param n_critic: number of times to update discriminator per generator update
        :param drift: drift penalty for the
                      (Used only if loss is wgan or wgan-gp)
        :param use_ema: boolean for whether to use exponential moving averages
        :param ema_decay: value of mu for ema
        :param device: device to run the GAN on (GPU / CPU)
        """

        from torch.optim import Adam
        from torch.nn import DataParallel

        # Create the Generator and the Discriminator
        self.gen = Generator(depth, latent_size).to(device)
        self.dis = Discriminator(depth, latent_size).to(device)

        # if code is to be run on GPU, we can use DataParallel:
        if device == th.device("cuda"):
            self.gen = DataParallel(self.gen)
            self.dis = DataParallel(self.dis)

        # state of the object
        self.latent_size = latent_size
        self.depth = depth
        self.use_ema = use_ema
        self.ema_decay = ema_decay
        self.n_critic = n_critic
        self.device = device
        self.drift = drift

        # define the optimizers for the discriminator and generator
        self.gen_optim = Adam(self.gen.parameters(), lr=learning_rate,
                              betas=(beta_1, beta_2), eps=eps)

        self.dis_optim = Adam(self.dis.parameters(), lr=learning_rate,
                              betas=(beta_1, beta_2), eps=eps)

        # define the loss function used for training the GAN
        self.loss = WGAN_GP(self.dis, self.drift, use_gp=True)

        if self.use_ema:

            # create a shadow copy of the generator
            self.gen_shadow = copy.deepcopy(self.gen)

            # updater function:
            self.ema_updater = update_average

            # initialize the gen_shadow weights equal to the
            # weights of gen
            self.ema_updater(self.gen_shadow, self.gen, beta=0)


    def __progressive_downsampling(self, real_batch, depth, alpha):
        """
        private helper for downsampling the original images in order to facilitate the
        progressive growing of the layers.
        :param real_batch: batch of real samples
        :param depth: depth at which training is going on
        :param alpha: current value of the fader alpha
        :return: real_samples => modified real batch of samples
        """

        from torch.nn import AvgPool2d
        from torch.nn.functional import interpolate

        # downsample the real_batch for the given depth
        down_sample_factor = int(np.power(2, self.depth - depth - 1))
        prior_downsample_factor = max(int(np.power(2, self.depth - depth)), 0)
        ds_real_samples = AvgPool2d(down_sample_factor)(real_batch)

        if depth > 0:
            prior_ds_real_samples = interpolate(AvgPool2d(prior_downsample_factor)(real_batch),
                                                scale_factor=2)
        else:
            prior_ds_real_samples = ds_real_samples

        # real samples are a combination of ds_real_samples and prior_ds_real_samples
        real_samples = (alpha * ds_real_samples) + ((1 - alpha) * prior_ds_real_samples)

        # return the so computed real_samples
        return real_samples

    def optimize_discriminator(self, noise, real_batch, depth, alpha):
        """
        performs one step of weight update on discriminator using the batch of data
        :param noise: input noise of sample generation
        :param real_batch: real samples batch
        :param depth: current depth of optimization
        :param alpha: current alpha for fade-in
        :return: current loss (Wasserstein loss)
        """

        real_samples = self.__progressive_downsampling(real_batch, depth, alpha)

        loss_val = 0
        for _ in range(self.n_critic):
            # generate a batch of samples
            fake_samples = self.gen(noise, depth, alpha).detach()
            
            loss = self.loss.dis_loss(real_samples, fake_samples, depth, alpha)

            # optimize discriminator
            self.dis_optim.zero_grad()
            loss.backward()
            self.dis_optim.step()

            loss_val += loss.item()

        return loss_val / self.n_critic

    def optimize_generator(self, noise, real_batch, depth, alpha):
        """
        performs one step of weight update on generator for the given batch_size
        :param noise: input random noise required for generating samples
        :param real_batch: batch of real samples
        :param depth: depth of the network at which optimization is done
        :param alpha: value of alpha for fade-in effect
        :return: current loss (Wasserstein estimate)
        """

        real_samples = self.__progressive_downsampling(real_batch, depth, alpha)

        # generate fake samples:
        fake_samples = self.gen(noise, depth, alpha)

        # TODO_complete:
        # Change this implementation for making it compatible for relativisticGAN
        loss = self.loss.gen_loss(real_samples, fake_samples, depth, alpha)

        # optimize the generator
        self.gen_optim.zero_grad()
        loss.backward()
        self.gen_optim.step()

        # if use_ema is true, apply ema to the generator parameters
        if self.use_ema:
            self.ema_updater(self.gen_shadow, self.gen, self.ema_decay)

        # return the loss value
        return loss.item()

    @staticmethod
    def create_grid(samples, scale_factor, img_file):
        """
        utility function to create a grid of GAN samples
        :param samples: generated samples for storing
        :param scale_factor: factor for upscaling the image
        :param img_file: name of file to write
        :return: None (saves a file)
        """
        from torchvision.utils import save_image
        from torch.nn.functional import interpolate

        # upsample the image
        if scale_factor > 1:
            samples = interpolate(samples, scale_factor=scale_factor)

        # save the images:
        save_image(samples, img_file, nrow=int(np.sqrt(len(samples))),
                   normalize=True, scale_each=True)

    def train(self, dataset, epochs, batch_sizes,
              fade_in_percentage, num_samples=16,
              start_depth=0, start_epoch=1, num_workers=3, feedback_factor=100,
              log_dir="./models/", sample_dir="./samples/", save_dir="./models/",
              checkpoint_factor=1):
        """
        Utility method for training the ProGAN. Note that you don't have to necessarily use this
        you can use the optimize_generator and optimize_discriminator for your own training routine.
        :param dataset: object of the dataset used for training.
                        Note that this is not the dataloader (we create dataloader in this method
                        since the batch_sizes for resolutions can be different)
        :param epochs: list of number of epochs to train the network for every resolution
        :param batch_sizes: list of batch_sizes for every resolution
        :param fade_in_percentage: list of percentages of epochs per resolution
                                   used for fading in the new layer
                                   not used for first resolution, but dummy value still needed.
        :param num_samples: number of samples generated in sample_sheet. def=36
        :param start_depth: start training from this depth. def=0
        :param num_workers: number of workers for reading the data. def=3
        :param feedback_factor: number of logs per epoch. def=100
        :param log_dir: directory for saving the loss logs. def="./models/"
        :param sample_dir: directory for saving the generated samples. def="./samples/"
        :param checkpoint_factor: save model after these many epochs.
                                  Note that only one model is stored per resolution.
                                  during one resolution, the checkpoint will be updated (Rewritten)
                                  according to this factor.
        :param save_dir: directory for saving the models (.pth files)
        :return: None (Writes multiple files to disk)
        """

        assert self.depth == len(batch_sizes), "batch_sizes not compatible with depth"

        #RELOAD
        gen_save_file = os.path.join(save_dir, "GAN_GEN_" + str(start_depth) + ".pth")
        dis_save_file = os.path.join(save_dir, "GAN_DIS_" + str(start_depth) + ".pth")
        gen_optim_save_file = os.path.join(save_dir,
                                            "GAN_GEN_OPTIM_" + str(start_depth)
                                            + ".pth")
        dis_optim_save_file = os.path.join(save_dir,
                                            "GAN_DIS_OPTIM_" + str(start_depth)
                                            + ".pth")
        
        #self.gen.load_state_dict(th.load(gen_save_file))
        #self.dis.load_state_dict(th.load(dis_save_file))
        #self.gen_optim.load_state_dict(th.load(gen_optim_save_file))
        #self.dis_optim.load_state_dict(th.load(dis_optim_save_file))

        # also save the shadow generator if use_ema is True
        if self.use_ema:
            gen_shadow_save_file = os.path.join(save_dir, "GAN_GEN_SHADOW_" +
                                                str(start_depth) + ".pth")
            #self.gen_shadow.load_state_dict(th.load(gen_shadow_save_file))
        #RELOAD

        # turn the generator and discriminator into train mode
        self.gen.train()
        self.dis.train()
        if self.use_ema:
            self.gen_shadow.train()

        # create a global time counter
        global_time = time.time()

        # create fixed_input for debugging
        fixed_input = th.randn(num_samples, self.latent_size).to(self.device)

        print("Starting the training process ... ")
        for current_depth in range(start_depth, self.depth):


            print("\n\nCurrently working on Depth: ", current_depth)
            current_res = np.power(2, current_depth + 1)
            print("Current resolution: %d x %d" % (current_res, current_res))
            
            if current_depth != START_DEPTH:
                start_epoch=1
            data = get_data_loader(dataset, batch_sizes[current_depth], num_workers)
            if (start_epoch > 1): ticker = len(range(1,start_epoch))* len(iter(data))
            else: ticker = 1
            print("Ticker", ticker)
            for epoch in range(start_epoch, epochs[current_depth] + 1):
                start = timeit.default_timer()  # record time at the start of epoch

                print("\nEpoch: %d" % epoch)
                total_batches = len(iter(data))

                fader_point = int((fade_in_percentage[current_depth] / 100)
                                  * epochs[current_depth] * total_batches)

                step = 0  # counter for number of iterations

                for (i, batch) in enumerate(data, 1):
                    # calculate the alpha for fading in the layers
                    alpha = ticker / fader_point if ticker <= fader_point else 1

                    # extract current batch of data for training
                    images = batch[0].to(self.device)

                    gan_input = th.randn(images.shape[0], self.latent_size).to(self.device)

                    # optimize the discriminator:
                    dis_loss = self.optimize_discriminator(gan_input, images,
                                                           current_depth, alpha)

                    # optimize the generator:
                    gen_loss = self.optimize_generator(gan_input, images, current_depth, alpha)

                    # provide a loss feedback
                    if i % int(total_batches / feedback_factor) == 0 or i == 1:
                        elapsed = time.time() - global_time
                        elapsed = str(datetime.timedelta(seconds=elapsed))
                        print("Elapsed: [%s]  batch: %d  d_loss: %f  g_loss: %f"
                              % (elapsed, i, dis_loss, gen_loss))

                        # also write the losses to the log file:
                        os.makedirs(log_dir, exist_ok=True)
                        log_file = os.path.join(log_dir, "loss_" + str(current_depth) + ".log")
                        with open(log_file, "a") as log:
                            log.write(str(step) + "\t" + str(dis_loss) +
                                      "\t" + str(gen_loss) + "\n")
                    # increment the alpha ticker and the step
                    ticker += 1
                    step += 1
                # create a grid of samples and save it
                os.makedirs(sample_dir, exist_ok=True)
                gen_img_file = os.path.join(sample_dir, "gen_" + str(current_depth) +
                                            "_" + str(epoch) + "_" +
                                            str(i) + ".png")

                # this is done to allow for more GPU space
                with th.no_grad():
                    self.create_grid(
                        samples=self.gen(
                            fixed_input,
                            current_depth,
                            alpha
                        ).detach() if not self.use_ema
                        else self.gen_shadow(
                            fixed_input,
                            current_depth,
                            alpha
                        ).detach(),
                        scale_factor=int(np.power(2, self.depth - current_depth - 1)),
                        img_file=gen_img_file,
                    )

                stop = timeit.default_timer()
                print("Time taken for epoch: %.3f secs" % (stop - start))
                print("ticker = ", ticker)
                if epoch % checkpoint_factor == 0 or epoch == 1 or epoch == epochs[current_depth]:
                    os.makedirs(save_dir, exist_ok=True)
                    gen_save_file = os.path.join(save_dir, "GAN_GEN_" + str(current_depth) + ".pth")
                    dis_save_file = os.path.join(save_dir, "GAN_DIS_" + str(current_depth) + ".pth")
                    gen_optim_save_file = os.path.join(save_dir,
                                                       "GAN_GEN_OPTIM_" + str(current_depth)
                                                       + ".pth")
                    dis_optim_save_file = os.path.join(save_dir,
                                                       "GAN_DIS_OPTIM_" + str(current_depth)
                                                       + ".pth")

                    th.save(self.gen.state_dict(), gen_save_file)
                    th.save(self.dis.state_dict(), dis_save_file)
                    th.save(self.gen_optim.state_dict(), gen_optim_save_file)
                    th.save(self.dis_optim.state_dict(), dis_optim_save_file)

                    # also save the shadow generator if use_ema is True
                    if self.use_ema:
                        gen_shadow_save_file = os.path.join(save_dir, "GAN_GEN_SHADOW_" +
                                                            str(current_depth) + ".pth")
                        th.save(self.gen_shadow.state_dict(), gen_shadow_save_file)

        # put the gen, shadow_gen and dis in eval mode
        self.gen.eval()
        self.dis.eval()
        if self.use_ema:
            self.gen_shadow.eval()

        print("Training completed ...")

In [11]:
import torch as th
import torchvision as tv
from torch.utils.data import TensorDataset
# select the device to be used for training
device = th.device("cuda" if th.cuda.is_available() else "cpu")

In [ ]:
import tensorflow as tf

In [13]:

path = "."
data_path = path + "/training_images"

# some parameters:
depth = 7
START_DEPTH = 0
START_EPOCH = 1
# hyper-parameters per depth (resolution)
num_epochs = [10, 25, 35, 50, 75, 100, 150]
fade_ins = [50, 50, 50, 50, 50, 50,50]
batch_sizes = depth*[20]
latent_size = 128
# get the data. Ignore the test data and their classes
dataset = tv.datasets.ImageFolder(root=data_path,transform=tv.transforms.ToTensor())
print(dataset)
print([dataset[i][0].shape for i in range(1)])
# dataset1 = np.load(path + '/Data/Train_5cm_norm.npz')
# dataset2 = dataset1['arr_0']
# print(dataset2.shape)
# dataset = TensorDataset(*dataset2)
# ======================================================================
# This line creates the PRO-GAN
# ======================================================================
# pro_gan = ConditionalProGAN(num_classes=10, depth=depth, 
#                                latent_size=latent_size, device=device)
pro_gan = ProGAN(depth=depth, 
                               latent_size=latent_size, device=device)
# ======================================================================

# ======================================================================
# This line trains the PRO-GAN
# ======================================================================
pro_gan.train(
    dataset=dataset,
    epochs=num_epochs,
    fade_in_percentage=fade_ins,
    batch_sizes=batch_sizes,
    log_dir=path + "/models/", sample_dir=path + "/samples/", save_dir= path + "/models/",
    start_depth=START_DEPTH,
    feedback_factor=10,
    start_epoch=START_EPOCH
)
# ======================================================================

Dataset ImageFolder
    Number of datapoints: 10000
    Root location: ./training_images
    StandardTransform
Transform: ToTensor()
[torch.Size([3, 128, 128])]
Starting the training process ... 


Currently working on Depth:  0
Current resolution: 2 x 2
Ticker 1

Epoch: 1
Elapsed: [0:00:07.953917]  batch: 1  d_loss: 1.358527  g_loss: -1.025170
Elapsed: [0:00:09.863514]  batch: 50  d_loss: -1.375130  g_loss: 2.600798
Elapsed: [0:00:11.821596]  batch: 100  d_loss: -1.971123  g_loss: 1.174121
Elapsed: [0:00:13.784848]  batch: 150  d_loss: -2.257158  g_loss: 1.994805
Elapsed: [0:00:15.757812]  batch: 200  d_loss: -1.753736  g_loss: 1.485612
Elapsed: [0:00:17.797737]  batch: 250  d_loss: -1.023956  g_loss: 0.887000
Elapsed: [0:00:19.919987]  batch: 300  d_loss: -0.871387  g_loss: 0.625090
Elapsed: [0:00:22.258585]  batch: 350  d_loss: -0.630591  g_loss: 0.497270
Elapsed: [0:00:24.261093]  batch: 400  d_loss: -0.550776  g_loss: 0.436087
Elapsed: [0:00:26.388425]  batch: 450  d_loss: -0.3122

Elapsed: [0:04:43.157167]  batch: 50  d_loss: 0.018389  g_loss: 0.077798
Elapsed: [0:04:45.041252]  batch: 100  d_loss: -0.080008  g_loss: 0.263681
Elapsed: [0:04:46.867120]  batch: 150  d_loss: -0.018589  g_loss: 0.059944
Elapsed: [0:04:48.685553]  batch: 200  d_loss: -0.007458  g_loss: 0.063142
Elapsed: [0:04:50.571781]  batch: 250  d_loss: -0.050699  g_loss: 0.227165
Elapsed: [0:04:52.499213]  batch: 300  d_loss: -0.082847  g_loss: 0.207220
Elapsed: [0:04:54.498968]  batch: 350  d_loss: -0.053322  g_loss: 0.070259
Elapsed: [0:04:56.421018]  batch: 400  d_loss: -0.065243  g_loss: -0.010705
Elapsed: [0:04:58.301960]  batch: 450  d_loss: 0.029731  g_loss: 0.356249
Elapsed: [0:05:00.206697]  batch: 500  d_loss: -0.072914  g_loss: 0.048720
Time taken for epoch: 28.186 secs
ticker =  5001


Currently working on Depth:  1
Current resolution: 4 x 4
Ticker 1

Epoch: 1
Elapsed: [0:05:09.577384]  batch: 1  d_loss: 2.270641  g_loss: 0.017553
Elapsed: [0:05:12.590697]  batch: 50  d_loss: 1.05542

Elapsed: [0:10:18.161973]  batch: 200  d_loss: -0.259844  g_loss: 0.348393
Elapsed: [0:10:20.933716]  batch: 250  d_loss: -0.114600  g_loss: 0.217215
Elapsed: [0:10:23.734231]  batch: 300  d_loss: -0.185148  g_loss: 0.162837
Elapsed: [0:10:26.551600]  batch: 350  d_loss: -0.165648  g_loss: 0.383260
Elapsed: [0:10:29.382267]  batch: 400  d_loss: -0.183015  g_loss: 0.230126
Elapsed: [0:10:32.181732]  batch: 450  d_loss: -0.253866  g_loss: 0.360400
Elapsed: [0:10:34.976525]  batch: 500  d_loss: -0.041960  g_loss: 0.150252
Time taken for epoch: 36.944 secs
ticker =  4501

Epoch: 10
Elapsed: [0:10:44.954864]  batch: 1  d_loss: -0.120720  g_loss: 0.177236
Elapsed: [0:10:48.666890]  batch: 50  d_loss: -0.096432  g_loss: 0.177085
Elapsed: [0:10:52.328467]  batch: 100  d_loss: -0.212464  g_loss: 0.196830
Elapsed: [0:10:56.097406]  batch: 150  d_loss: -0.051673  g_loss: 0.328278
Elapsed: [0:10:59.426878]  batch: 200  d_loss: -0.015798  g_loss: 0.114169
Elapsed: [0:11:03.001943]  batch: 250  d_lo

Elapsed: [0:16:29.949060]  batch: 400  d_loss: -0.155367  g_loss: 0.314983
Elapsed: [0:16:33.090598]  batch: 450  d_loss: 0.051540  g_loss: 0.221091
Elapsed: [0:16:36.269662]  batch: 500  d_loss: -0.089193  g_loss: 0.190424
Time taken for epoch: 38.175 secs
ticker =  9001

Epoch: 19
Elapsed: [0:16:46.676976]  batch: 1  d_loss: 0.008240  g_loss: 0.051423
Elapsed: [0:16:49.815106]  batch: 50  d_loss: 0.064033  g_loss: 0.065273
Elapsed: [0:16:53.084525]  batch: 100  d_loss: 0.059366  g_loss: 0.168941
Elapsed: [0:16:56.401689]  batch: 150  d_loss: -0.011071  g_loss: 0.184325
Elapsed: [0:16:59.784539]  batch: 200  d_loss: -0.186043  g_loss: 0.342716
Elapsed: [0:17:03.037068]  batch: 250  d_loss: 0.044187  g_loss: 0.080716
Elapsed: [0:17:06.455585]  batch: 300  d_loss: -0.094477  g_loss: 0.332080
Elapsed: [0:17:09.978284]  batch: 350  d_loss: -0.027604  g_loss: 0.232553
Elapsed: [0:17:13.336878]  batch: 400  d_loss: -0.122804  g_loss: 0.250613
Elapsed: [0:17:16.745128]  batch: 450  d_loss: 0

Time taken for epoch: 45.935 secs
ticker =  1001

Epoch: 3
Elapsed: [0:23:05.936927]  batch: 1  d_loss: -0.181198  g_loss: 0.364263
Elapsed: [0:23:09.422366]  batch: 50  d_loss: -0.215988  g_loss: 0.230436
Elapsed: [0:23:12.987432]  batch: 100  d_loss: -0.157882  g_loss: 0.240106
Elapsed: [0:23:16.603893]  batch: 150  d_loss: -0.036479  g_loss: 0.217309
Elapsed: [0:23:20.201865]  batch: 200  d_loss: -0.071626  g_loss: 0.331861
Elapsed: [0:23:23.743869]  batch: 250  d_loss: 0.049273  g_loss: 0.176732
Elapsed: [0:23:27.296339]  batch: 300  d_loss: -0.053481  g_loss: 0.379103
Elapsed: [0:23:30.834893]  batch: 350  d_loss: -0.189251  g_loss: 0.393229
Elapsed: [0:23:34.382361]  batch: 400  d_loss: -0.046732  g_loss: 0.261065
Elapsed: [0:23:37.950307]  batch: 450  d_loss: -0.177254  g_loss: 0.194913
Elapsed: [0:23:41.547307]  batch: 500  d_loss: -0.072497  g_loss: 0.328359
Time taken for epoch: 44.687 secs
ticker =  1501

Epoch: 4
Elapsed: [0:23:50.889246]  batch: 1  d_loss: -0.278530  g_los

...

Elapsed: [1 day, 1:22:10.250345]  batch: 400  d_loss: -0.032602  g_loss: 0.330644
Elapsed: [1 day, 1:22:49.087701]  batch: 450  d_loss: -0.130776  g_loss: 0.462800
Elapsed: [1 day, 1:23:27.578562]  batch: 500  d_loss: -0.263249  g_loss: 0.573325
Time taken for epoch: 396.033 secs
ticker =  57001

Epoch: 115
Elapsed: [1 day, 1:23:45.666726]  batch: 1  d_loss: 0.115810  g_loss: 0.061311
Elapsed: [1 day, 1:24:13.027415]  batch: 50  d_loss: 0.103409  g_loss: -0.137037
Elapsed: [1 day, 1:24:40.697996]  batch: 100  d_loss: -0.293041  g_loss: 0.204121
Elapsed: [1 day, 1:25:08.169663]  batch: 150  d_loss: -0.044587  g_loss: 0.137129
Elapsed: [1 day, 1:25:35.477599]  batch: 200  d_loss: 0.076299  g_loss: -0.159976
Elapsed: [1 day, 1:26:02.676718]  batch: 250  d_loss: 0.025309  g_loss: 0.088826
Elapsed: [1 day, 1:26:29.776229]  batch: 300  d_loss: -0.049574  g_loss: 0.372916
Elapsed: [1 day, 1:26:56.863929]  batch: 350  d_loss: -0.060775  g_loss: 0.476003
Elapsed: [1 day, 1:27:23.880986]  batch:

Elapsed: [1 day, 2:10:04.900585]  batch: 150  d_loss: -0.001904  g_loss: 0.230042
Elapsed: [1 day, 2:10:31.900769]  batch: 200  d_loss: -0.005804  g_loss: -0.062081
Elapsed: [1 day, 2:10:58.916009]  batch: 250  d_loss: 0.134343  g_loss: 0.025283
Elapsed: [1 day, 2:11:25.921493]  batch: 300  d_loss: -0.191755  g_loss: 0.312973
Elapsed: [1 day, 2:11:52.823910]  batch: 350  d_loss: 0.111555  g_loss: 0.184752
Elapsed: [1 day, 2:12:19.716978]  batch: 400  d_loss: -0.117635  g_loss: 0.642344
Elapsed: [1 day, 2:14:51.717611]  batch: 450  d_loss: 0.040024  g_loss: 0.537198
Elapsed: [1 day, 2:18:03.100704]  batch: 500  d_loss: 0.104055  g_loss: -0.281895
Time taken for epoch: 570.780 secs
ticker =  61501

Epoch: 124
Elapsed: [1 day, 2:18:12.508041]  batch: 1  d_loss: -0.049527  g_loss: 0.270488
Elapsed: [1 day, 2:18:39.096927]  batch: 50  d_loss: 0.046193  g_loss: 0.656962
Elapsed: [1 day, 2:19:06.715847]  batch: 100  d_loss: -0.309023  g_loss: 0.707811
Elapsed: [1 day, 2:19:34.691582]  batch: 

Elapsed: [1 day, 3:29:37.025236]  batch: 500  d_loss: -0.072391  g_loss: 0.263413
Time taken for epoch: 775.837 secs
ticker =  65501

Epoch: 132
Elapsed: [1 day, 3:29:58.640366]  batch: 1  d_loss: 0.012102  g_loss: 0.007364
Elapsed: [1 day, 3:30:49.681216]  batch: 50  d_loss: 0.094317  g_loss: 0.165659
Elapsed: [1 day, 3:31:52.618139]  batch: 100  d_loss: -0.126690  g_loss: 0.272588
Elapsed: [1 day, 3:32:58.187222]  batch: 150  d_loss: 0.037352  g_loss: 0.365715
Elapsed: [1 day, 3:34:06.337693]  batch: 200  d_loss: -0.083908  g_loss: 0.437012
Elapsed: [1 day, 3:35:11.965392]  batch: 250  d_loss: 0.027564  g_loss: 0.179753
Elapsed: [1 day, 3:36:20.157505]  batch: 300  d_loss: -0.069062  g_loss: 0.258912
Elapsed: [1 day, 3:37:23.374550]  batch: 350  d_loss: -0.131374  g_loss: 0.403990
Elapsed: [1 day, 3:38:47.586965]  batch: 400  d_loss: 0.034936  g_loss: 0.124690
Elapsed: [1 day, 3:39:58.792544]  batch: 450  d_loss: -0.292393  g_loss: 0.332862
Elapsed: [1 day, 3:41:11.946484]  batch: 50

Elapsed: [1 day, 4:36:30.611864]  batch: 250  d_loss: -0.168855  g_loss: 0.368229
Elapsed: [1 day, 4:36:57.616359]  batch: 300  d_loss: -0.062365  g_loss: 0.463297
Elapsed: [1 day, 4:37:24.671078]  batch: 350  d_loss: -0.327568  g_loss: 0.457292
Elapsed: [1 day, 4:37:51.654029]  batch: 400  d_loss: -0.085582  g_loss: 0.528141
Elapsed: [1 day, 4:38:18.720011]  batch: 450  d_loss: -0.123315  g_loss: 0.440099
Elapsed: [1 day, 4:38:45.743187]  batch: 500  d_loss: -0.120514  g_loss: 0.479952
Time taken for epoch: 282.401 secs
ticker =  70001

Epoch: 141
Elapsed: [1 day, 4:38:58.097622]  batch: 1  d_loss: 0.051492  g_loss: 0.122134
Elapsed: [1 day, 4:39:24.171823]  batch: 50  d_loss: -0.104816  g_loss: 0.360802
Elapsed: [1 day, 4:39:51.170166]  batch: 100  d_loss: 0.018789  g_loss: -0.030378
Elapsed: [1 day, 4:40:18.232843]  batch: 150  d_loss: -0.127154  g_loss: 0.458685
Elapsed: [1 day, 4:40:45.246369]  batch: 200  d_loss: -0.160477  g_loss: 0.358790
Elapsed: [1 day, 4:41:12.236746]  batch


Epoch: 149
Elapsed: [1 day, 5:13:24.070204]  batch: 1  d_loss: -0.280204  g_loss: 0.556040
Elapsed: [1 day, 5:13:45.916221]  batch: 50  d_loss: 0.001867  g_loss: 0.086287
Elapsed: [1 day, 5:14:08.602118]  batch: 100  d_loss: 0.053681  g_loss: 0.497668
Elapsed: [1 day, 5:14:31.286700]  batch: 150  d_loss: -0.267893  g_loss: 0.483749
Elapsed: [1 day, 5:14:54.001423]  batch: 200  d_loss: -0.110940  g_loss: 0.687385
Elapsed: [1 day, 5:15:16.766541]  batch: 250  d_loss: 0.107022  g_loss: -0.355600
Elapsed: [1 day, 5:15:39.456154]  batch: 300  d_loss: -0.173507  g_loss: 0.490215
Elapsed: [1 day, 5:16:02.209115]  batch: 350  d_loss: 0.015733  g_loss: 0.136132
Elapsed: [1 day, 5:16:24.932029]  batch: 400  d_loss: -0.109720  g_loss: 0.163083
Elapsed: [1 day, 5:16:47.623327]  batch: 450  d_loss: -0.054447  g_loss: -0.010284
Elapsed: [1 day, 5:17:10.387563]  batch: 500  d_loss: 0.026237  g_loss: 0.304077
Time taken for epoch: 243.103 secs
ticker =  74501

Epoch: 150
Elapsed: [1 day, 5:17:27.7277

In [78]:
for file in os.listdir('dataset/passive_margin'):
        tab = np.load('dataset/passive_margin/'+file)
        tab = (tab - np.mean(tab))/np.std(tab)
        tab2=np.zeros((tab.shape[0],tab.shape[1],3))
        tab2[:,:,0]=tab
        tab2[:,:,1]=tab
        tab2[:,:,2]=tab
        name=file.split('.')[0]
        cv2.imwrite(f'training_images/passive_margin/{name}.png',
                    np.uint8((tab2-np.min(tab2))/(np.max(tab2)-np.min(tab2))*255))

In [2]:
import cv2
import numpy as np
import os

In [10]:
cv2.imwrite('image.jpg',np.zeros((128,128,3)))

True

In [17]:
189*1.15

217.35

In [ ]:
()

In [16]:
(110000 -48400)/(7*110000)

0.08